In [22]:
# configuration
property_label = "P19"
# should be "subjects" or "objects"
orientation = "objects"

In [26]:
import csv
import statistics

node_subject_props = {}
node_object_props = {}
dis_class = set()
tsv_file = open(f'../output/instance_prediction_datasets/{property_label}.{orientation}.1250.tsv',encoding="utf-8")
read_tsv = csv.reader(tsv_file, delimiter="\t")
next(read_tsv)
all_node_subject_props = []
all_node_object_props = []
for line in read_tsv:
    dis_class.add(line[0])
#     if line[5] == "subject":
#         all_node_subject_props.append(line[2])
#         if line[0] in node_subject_props:
#             node_subject_props[line[0]].append(line[2])
#         else:
#             node_subject_props[line[0]] = [line[2]]
#     elif line[5] == "object":
#         all_node_object_props.append(line[2])
#         if line[0] in node_object_props:
#             node_object_props[line[0]].append(line[2])
#         else:
#             node_object_props[line[0]] = [line[2]]

# print("node subject props size: ", len(all_node_subject_props)/len(dis_class))
# print("distinct node subject props: ", len(set(all_node_subject_props)))
# print("node object props size: ", len(all_node_object_props)/len(dis_class))
# print("distinct node object props: ", len(set(all_node_object_props)))

print("distinct classes: ", len(dis_class))

print("making predictions...")
#guesses = predictWithConstraints(dis_class, node_subject_props, node_object_props)
guesses = predictWithEmbeddings(dis_class)

print("retrieving correct answers...")
correct_answers = getCorrectAnswers(list(dis_class))

print("retrieving scores...")
scores = []
guess_efficacy = {}
for elem in guesses:
    this_score = score(guesses[elem], correct_answers[elem])
    scores.append(this_score)
    if guesses[elem] in guess_efficacy:
        guess_efficacy[guesses[elem]][0] += 1
        guess_efficacy[guesses[elem]][1] += this_score
    else:
        guess_efficacy[guesses[elem]] = [1, this_score]
print("final avg. score: ", statistics.mean(scores))
for k,v in sorted(guess_efficacy.items(), key=lambda p:p[1], reverse=True):
    print(k,v[0],(v[1]/v[0]))

distinct classes:  1260
making predictions...
retrieving correct answers...
retrieving scores...
final avg. score:  0.002425079365079365
Q21278897 266 0.0
Q4533081 166 0.0
Q18142 116 0.0
Q607241 84 0.005833333333333333
Q394067 67 0.0
Q82117 55 0.0
Q7366 50 0.0
Q81058955 32 0.0
Q2384959 31 0.0
Q17633526 25 0.0
Q33388604 25 0.0
Q521972 23 0.0
Q28640 18 0.0
Q1344 18 0.0
Q3802482 17 0.0
Q231002 16 0.0
Q83328 15 0.0
Q2588070 12 0.0
Q4209223 12 0.0
Q899409 12 0.0
Q334383 11 0.0
Q20867284 10 0.04096000000000001
Q2775 9 0.0
Q1707610 8 0.0
Q22947 8 0.0
Q17624511 7 0.09799999999999998
Q1365179 5 0.0
Q14839548 5 0.0
Q4989906 5 0.0
Q3678424 4 0.36749999999999994
Q2623243 4 0.0
Q6502866 4 0.0
Q15632166 4 0.0
Q33513247 4 0.0
Q47012103 4 0.0
Q16970 4 0.0
Q160742 4 0.0
Q4057659 4 0.0
Q47150325 4 0.0
Q53060 4 0.0
Q1267889 4 0.0
Q25379 3 0.0
Q11879590 3 0.0
Q207628 3 0.0
Q2116450 3 0.0
Q1575478 3 0.0
Q29586452 3 0.0
Q744099 3 0.0
Q4156067 3 0.0
Q1406988 2 0.0
Q925455 2 0.0
Q3931974 2 0.0
Q190157 2 0.0
Q

In [16]:
# score P31 datasets
import csv

direct_subclasses = {}
direct_superclasses = {}

tsv_file = open('../derived.P279.tsv/derived.P279.tsv',encoding="utf-8")
read_tsv = csv.reader(tsv_file, delimiter="\t")
next(read_tsv)
for line in read_tsv:
    if line[1] not in direct_superclasses:
        direct_superclasses[line[1]] = [line[3]]
    else:
        direct_superclasses[line[1]].append(line[3])
    if line[3] not in direct_subclasses:
        direct_subclasses[line[3]] = [line[1]]
    else:
        direct_subclasses[line[3]].append(line[1])

In [13]:
import mmap

def getCorrectAnswers(candidates):
    answers = {}
#     with open('../derived.P31.tsv/derived.P31.tsv',encoding="utf-8") as f:
#         s = mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ)
#         while len(candidates) != 0:
#             line = s.readline()
#             for candidate in candidates:
#                 if str.encode("\t"+candidate+"\t") in line:
#                     str1 = line.decode("utf-8")
#                     split = str1.split("\t")
#                     answer = split[-1].split("\n")[0]
#                     answers[candidate] = answer
#                     candidates.remove(candidate)
    tsv_file = open(f'../output/allConstraintsAnalysis_Final/instances/instances.{orientation}.{property_label}.tsv', encoding="utf-8")
    read_tsv = csv.reader(tsv_file, delimiter="\t")
    for line in read_tsv:
        answers[line[0]] = line[1]
    return answers

In [14]:
def score (guess, correct):
    if guess == correct: 
        return 1
    else:
        subclasses = []
        superclasses = []
        if guess in direct_subclasses:
            subclasses = direct_subclasses[guess]
        if guess in direct_superclasses:
            superclasses = direct_superclasses[guess]
        for i in range(1,6):
            new_subclasses = []
            new_superclasses = []
            for subc in subclasses:
                if subc == correct:
                    if i <= 3:
                        return .7**i
                    else:
                        return .8**i
                else:
                    if subc in direct_subclasses:
                        for new_sub in direct_subclasses[subc]:
                            new_subclasses.append(new_sub)
            for supc in superclasses:
                if supc == correct:
                    if i <= 3:
                        return .7**i
                    else:
                        return .8**i
                else:
                    if supc in direct_superclasses:
                        for new_sup in direct_superclasses[supc]:
                            new_superclasses.append(new_sup)
            subclasses = new_subclasses
            superclasses = new_superclasses
    return 0

In [8]:
# load constraint values
typeConstraints = {}
tsv_file = open(f'../data/property_constraints/all_type_constraints.tsv', encoding="utf-8")
read_tsv = csv.reader(tsv_file, delimiter="\t")
next(read_tsv)
for line in read_tsv:
    if line[0].replace("http://www.wikidata.org/entity/","") in typeConstraints:
        typeConstraints[line[0].replace("http://www.wikidata.org/entity/","")].append(line[2].replace("http://www.wikidata.org/entity/",""))
    else:
        typeConstraints[line[0].replace("http://www.wikidata.org/entity/","")] = [line[2].replace("http://www.wikidata.org/entity/","")]

valueTypeConstraints = {}
tsv_file = open(f'../data/property_constraints/all_value_type_constraints.tsv', encoding="utf-8")
read_tsv = csv.reader(tsv_file, delimiter="\t")
next(read_tsv)
for line in read_tsv:
    if line[0].replace("http://www.wikidata.org/entity/","") in valueTypeConstraints:
        valueTypeConstraints[line[0].replace("http://www.wikidata.org/entity/","")].append(line[2].replace("http://www.wikidata.org/entity/",""))
    else:
        valueTypeConstraints[line[0].replace("http://www.wikidata.org/entity/","")] = [line[2].replace("http://www.wikidata.org/entity/","")]

In [39]:
def predictWithConstraints(dis_class, node_subject_props, node_object_props):
    final_guesses = {}
    for node in dis_class:
        all_class_votes = {}
        if node in node_subject_props:
            subject_props = node_subject_props[node]
            for prop in subject_props:
                if prop in typeConstraints:
                    for constraint in typeConstraints[prop]:
                        if constraint in all_class_votes:
                            all_class_votes[constraint] += 1
                        else:
                            all_class_votes[constraint] = 1
        if node in node_object_props:
            object_props = node_object_props[node]
            for prop in object_props:
                if prop in valueTypeConstraints:
                    for constraint in valueTypeConstraints[prop]:
                        if constraint in all_class_votes:
                            all_class_votes[constraint] += 1
                        else:
                            all_class_votes[constraint] = 1
        
        max_votes = []
        max_score = 0
        for vote in all_class_votes:
            if all_class_votes[vote] == max_score:
                max_votes.append(vote)
            if all_class_votes[vote] > max_score:
                max_votes = []
                max_votes.append(vote)
                max_score = all_class_votes[vote]
                
        elected = ""
        max_score = 0
        for vote in max_votes:
            if vote in instance_counts:
                if instance_counts[vote] > max_score:
                    elected = vote
                    max_score = instance_counts[vote]
                
        final_guesses[node] = elected
    
#     values_map = {}
#     values = final_guesses.values()
#     for value in values:
#         if value in values_map:
#             values_map[value] += 1
#         else:
#             values_map[value] = 1
#     print(values_map)
    
    return final_guesses

In [13]:
# add instance counts for tiebreaker
import csv

instance_counts = {}
tsv_file = open(f'../statistics.Pinstance_count.tsv/statistics.Pinstance_count.tsv', encoding="utf-8")
read_tsv = csv.reader(tsv_file, delimiter="\t")
next(read_tsv)
for line in read_tsv:
    instance_counts[line[0]] = int(line[2])

In [24]:
import json
from gensim.models import KeyedVectors
import tempfile
import pandas as pd
import numpy as np
import h5py, torch
from torchbiggraph.model import ComplexDiagonalDynamicOperator, DotComparator, CosComparator

vector_dimension = 100

# prepare embeddings data

relation_names_list = json.load(open("/out/embeddings/dynamic_rel_names.json"))
entity_names_list = json.load(open("/out/embeddings/entity_names_all_0.json"))
print("loaded entities and rels")
prop_count = len(relation_names_list)

# operators
operator_lhs = ComplexDiagonalDynamicOperator(vector_dimension, prop_count)
operator_rhs = ComplexDiagonalDynamicOperator(vector_dimension, prop_count)
comparator = DotComparator()
cos_comparator = CosComparator()
print("loading model")
with h5py.File("/out/embeddings/model.v600.h5", "r") as hf:
    operator_state_dict_lhs = {
        "real": torch.from_numpy(hf["model/relations/0/operator/lhs/real"][...]),
        "imag": torch.from_numpy(hf["model/relations/0/operator/lhs/imag"][...]),
    }
    operator_state_dict_rhs = {
        "real": torch.from_numpy(hf["model/relations/0/operator/rhs/real"][...]),
        "imag": torch.from_numpy(hf["model/relations/0/operator/rhs/imag"][...]),
    }
print("loaded model")
    
operator_lhs.load_state_dict(operator_state_dict_lhs)
operator_rhs.load_state_dict(operator_state_dict_rhs)
print("loaded state dict")

entity_to_index = {}
for i, entity in enumerate(entity_names_list):
    entity_to_index[entity] = i
    

rel_index = {}
for i, rel in enumerate(relation_names_list):
    rel_index[rel] = i
    
# Load the embeddings
#with h5py.File("/out/embeddings/embeddings_all_0.v600.h5", "r") as hf:
hf = np.memmap("/out/embeddings/wikidata-20210215-dwd-v2-similarity-embed.2021-10-03T12_14.complex.np.mmap", mode='r',
              shape=(212010680,100))
arnold_embedding = torch.from_numpy(hf)
print("loaded embeddings")
print(np.shape(arnold_embedding))
print(arnold_embedding[0])

def get_embed(head, relation=None):
    ''' This function generate the embeddings for the tail entities:
            Head entities: Obtained from the model
            Head + relation: Obtained using torch
        :param head: subject Qnode
        :param relation: optional property
    '''
    if relation is None:
        return arnold_embedding[entity_to_index[head], :].detach().numpy()
    return  operator_lhs(
                arnold_embedding[entity_to_index[head], :].view(1, vector_dimension),
                torch.tensor([rel_index[relation]])
            ).detach().numpy()[0]

def kgtk_most_similar(
    vectors,
    positive,
    relation_label="similarity_score",
    add_label_description=False,
    output_path=None,
    topn=25,
):
    """
    find topn similar Qnodes, add label and decription for the Qnodes
    
    :param vectors: vector space loaded into gensim KeyedVectors model
    :param positive: vector(s) or Qnode(s) to find similar entities for
    :param relation_label: name of the property to be used for the output file
    :param add_label_description: boolean parameter to add label and description for matched entities
    :param output_path: path to store the output file
    :param topn: desirednumber of similar entities
    """
    result = []
    if add_label_description:
        fp = tempfile.NamedTemporaryFile(
            mode="w", suffix=".tsv", delete=False, encoding="utf-8"
        )
        fp.write("node1\tlabel\tnode2\n")
        for (qnode, similarity) in vectors.most_similar(positive=positive, topn=topn):
            fp.write("{}\t{}\t{}\n".format(qnode, relation_label, similarity))
        filename = fp.name
        fp.close()
        
        os.environ["_temp_file"] = filename

        result = !$kypher -i label -i description -i "$_temp_file" --as sim \
--match 'sim: (n1)-[]->(similarity), label: (n1)-[]->(lab), description: (n1)-[]->(des)' \
--return 'distinct n1 as node1, similarity as node2, "similarity" as label, lab as `node1;label`, des as `node1;description`' \
--order-by 'cast(similarity, float) desc' 
        
        os.remove(filename)
        
    else:
        result.append("node1\tlabel\tnode2\n")
        for (qnode, similarity) in vectors.most_similar(positive=positive, topn=topn):
            result.append("{}\t{}\t{}\n".format(qnode, relation_label, similarity))

    if output_path:
        handle = open(output_path, "w")
        for line in result:
            handle.write(line)
            handle.write("\n")
        handle.close()
    else:
        columns = result[0].split("\t")
        data = []
        for line in result[1:]:
            data.append(line.split("\t"))
        return pd.DataFrame(data, columns=columns)
    
ge_vectors = KeyedVectors.load_word2vec_format(f"/out/embeddings/embeddings.{property_label}.{orientation}.w2v", binary=False)
print(ge_vectors[0])
print(ge_vectors[1])

loaded entities and rels
loading model
loaded model
loaded state dict
loaded embeddings
torch.Size([212010680, 100])
tensor([ 72, 167, 196, 188, 212,  93,  94, 191, 159, 240, 128,  61,  17, 222,
          8, 190,  57,  44,  98,  61, 237, 146, 204, 189, 225, 160, 170,  62,
         85, 136, 135, 190,  15, 233,  60,  61,  87, 130,   1, 190,  99, 186,
        251,  61, 177, 243, 204,  60, 187,  24, 205, 190,   2, 215, 113, 190,
         97,  37, 128, 190, 144,  44,  37,  62, 249, 145, 120,  62, 178, 155,
        156, 190, 183, 200,  15, 190,  96, 202, 190,  60,  44, 130,  31, 190,
         59, 101,  78, 190,   0, 122, 173,  61,  62,  69,  27, 190, 102, 166,
        146,  61], dtype=torch.uint8)
[-0.20318848  0.38883767 -0.13042542  0.40719104  0.0073998   0.02770344
 -0.3692729  -0.07117436 -0.45070335  0.2896174  -0.11128167  0.1264658
  0.08620919 -0.17981598  0.18693425 -0.12659058 -0.5986237  -0.02160399
  0.1482508   0.4913708  -0.1796804   0.17583902  0.06972955  0.07167171
  0.0237

In [23]:
def predictWithEmbeddings(inputs):
    final_guesses = {}
    for cla in inputs:
        _vector = get_embed(cla, 'P31')
        final_guesses[cla] = kgtk_most_similar(ge_vectors, positive=[_vector], topn=1)["node1"].array[0]
    return final_guesses